In [1]:
from DataLoader import DataLoader
from DataCleaner import DataCleaner
from DataSplit import DataSplit, DataSet
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline as SparkPipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import PipelineModel
from Pipeline import PreProcessingPipeline
from pyspark.sql.functions import upper, initcap
from Config import Config

/Users/julius/repos/BD-Project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

val_data = DataLoader(DataSplit.VALIDATION).data
test_data = DataLoader(DataSplit.TEST).data

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 10:27:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
val_preprocessing_pipeline = PreProcessingPipeline(DataSplit.VALIDATION)
test_preprocessing_pipeline = PreProcessingPipeline(DataSplit.TEST)



In [4]:
val_df = val_preprocessing_pipeline.run()
test_df = test_preprocessing_pipeline.run()

In [5]:
model = PipelineModel.load(Config.MODEL_PATH)

In [6]:
val_preds = model.transform(val_df)
test_preds = model.transform(test_df)

24/03/14 10:27:35 WARN StringIndexerModel: Input column label does not exist during transformation. Skip StringIndexerModel for this column.
24/03/14 10:27:35 WARN StringIndexerModel: Input column label does not exist during transformation. Skip StringIndexerModel for this column.


In [7]:
val_preds.select("prediction").show(5)

+----------+
|prediction|
+----------+
|       0.0|
|       0.0|
|       0.0|
|       1.0|
|       0.0|
+----------+
only showing top 5 rows



In [8]:
# add label column, convering from 0,1 to False, True
val_preds = val_preds.withColumn("label", val_preds["prediction"].cast("boolean").cast("string"))
test_preds = test_preds.withColumn("label", test_preds["prediction"].cast("boolean").cast("string"))

In [9]:
val_preds = val_preds.select(initcap("label"))
test_preds = test_preds.select(initcap("label"))

In [10]:
val_preds.show(5)

+--------------+
|initcap(label)|
+--------------+
|         False|
|         False|
|         False|
|          True|
|         False|
+--------------+
only showing top 5 rows



In [11]:
val_preds.write.csv(f'{Config.PREDICTIONS_PATH}{Config.VALIDATION_PREDICTIONS_NAME}', mode="overwrite", header=False)
test_preds.write.csv(f'{Config.PREDICTIONS_PATH}{Config.TEST_PREDICTIONS_NAME}', mode="overwrite", header=False)
                  